## Data Preprocessing Part

In [3]:
# imports
import pandas as pd
from PIL import Image
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import os

In [11]:
# check dimensions of images

min_width = 10000000
max_width = 0

min_height = 10000000
max_height = 0

for file in os.listdir('data/train'):
    if file == '_annotations.csv':
        continue
    img = Image.open('data/train/' + file)
    width, height = img.size
    if width < min_width:
        min_width = width
    if width > max_width:
        max_width = width
    if height < min_height:
        min_height = height
    if height > max_height:
        max_height = height

print(min_width, max_width, min_height, max_height)

372 416 372 416


In [ ]:
# 372 416 372 416